In [18]:
# Import necessary libraries
from ultralytics import YOLO

In [19]:
model = YOLO("weights/bestV4.pt")

In [20]:
results = model.predict('training images/IMG_0500.JPEG', conf=0.05)
results[0].show()


image 1/1 c:\Users\samad\Coding\Tuwaiq\capstone-project-team01\training images\IMG_0500.JPEG: 640x640 16 blacks, 32 emptys, 14 whites, 324.1ms
Speed: 14.7ms preprocess, 324.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


In [21]:
# Function to get the board coordinates
def get_board_coordinates(xyxy):
    min_x = xyxy[:, 0].min().item()
    max_x = xyxy[:, 2].max().item()
    min_y = xyxy[:, 1].min().item()
    max_y = xyxy[:, 3].max().item()
    return min_x - 5, min_y - 5, max_x + 5, max_y + 5

In [22]:
xyxy = results[0].boxes.xyxy
min_x ,min_y ,max_x ,max_y = get_board_coordinates(xyxy)
print(f"Crop coordinates: ({min_x}, {min_y}), ({max_x}, {max_y})")

Crop coordinates: (158.50347900390625, 147.17041015625), (1539.111083984375, 1610.830810546875)


In [23]:
from PIL import Image

In [31]:
# Open the image using Pillow
image_path = 'training images/IMG_0500.JPEG'
img = Image.open(image_path)

# Crop the image
cropped_img = img.crop((min_x, min_y, max_x, max_y))

# Save the cropped image
cropped_img.save('cropped_image.JPEG')
print("Cropped image saved as 'cropped_image.JPEG'.")

Cropped image saved as 'cropped_image.JPEG'.


In [32]:
# predict based on cropped
results = model.predict(cropped_img, conf=0.05)
results[0].show()


0: 640x608 16 blacks, 32 emptys, 15 whites, 255.9ms
Speed: 21.6ms preprocess, 255.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 608)


In [33]:
# Map detections to cells (reversed grid)
def map_detections_to_spaces(boxes, spaces, classes, frame_shape, grid_rows, grid_cols):
    # Initialize all spaces to "initial"
    occupancy = {space: "initial" for space in spaces}
    index = 0
    
    for box in boxes:
        # Calculate the center of the box
        x_center = (box[0] + box[2]) / 2
        y_center = (box[1] + box[3]) / 2
        
        # Calculate row and column indices
        row = int(y_center * grid_rows / frame_shape[0])
        col = int(x_center * grid_cols / frame_shape[1])
        
        # Clamp row and column indices to valid grid bounds
        row = max(0, min(row, grid_rows - 1))  # Ensure 0 <= row < grid_rows
        col = max(0, min(col, grid_cols - 1))  # Ensure 0 <= col < grid_cols
        
        # Map to a space identifier, reversed for rows
        space = f"{chr(72 - row)}{col + 1}"  # Reverse row index to start from 'H' (72)
        occupancy[space] = classes[index]
        
        index += 1
    return occupancy
    

In [34]:
grid_rows = grid_cols = 8

# Create a grid for spaces (reversed order)
def create_grid(grid_rows, grid_cols):
    spaces = []
    for i in range(grid_rows):
        for j in range(grid_cols):
            spaces.append(f"{chr(72 - i)}{j + 1}")  # Start from 'H' (72 in ASCII)
    return spaces
spaces = create_grid(grid_rows, grid_cols)

In [35]:
boxes = results[0].boxes.xyxy.cpu().numpy() if results else []

predicted_classes = results[0].boxes.cls  # `cls` contains the class indices
class_names = model.names  # Assuming the model has a `names` attribute for class labels
predicted_class_names = [class_names[int(cls_idx)] for cls_idx in predicted_classes]

In [36]:
occupancy = map_detections_to_spaces(boxes, spaces, predicted_class_names, cropped_img.size, 8, 8)
occupancy['A8']

'initial'

In [ ]:
def map_occupancy_to_board_status(occupancy):
    # Mapping rows and columns to a 2D array
    rows = ['H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']
    columns = ['1', '2', '3', '4', '5', '6', '7', '8']

    new_board_status = []

    for row in rows:
        row_data = []
        for col in columns:
            key = f"{row}{col}"
            cell_value = occupancy[key]
            row_data.append(cell_value)
        new_board_status.append(row_data)
    
    return new_board_status

new_board_status = map_occupancy_to_board_status(occupancy)

# Print the resulting 2D array
for row in new_board_status:
    print(row)

['black', 'black', 'black', 'black', 'black', 'black', 'black', 'initial']
['black', 'black', 'black', 'black', 'black', 'black', 'black', 'initial']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'initial']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'initial']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'white', 'initial']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'initial']
['white', 'white', 'white', 'white', 'white', 'white', 'white', 'empty']
['white', 'white', 'white', 'white', 'white', 'white', 'white', 'initial']
